Import required modules

In [2]:
import cv2
import gc
import glob
import json
import math
import numpy as np
import os
import patient_data
import pydicom
import tensorflow as tf

from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import backend as K
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Access the folder path for the cancer and the non-cancer images

In [2]:
all_paths = json.loads(open("./paths.json").read())

personal_path = all_paths['personal_path']
cancerous_path = personal_path + all_paths['cancerous_path']

Configure GPUs is applicable

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print('Num GPUs Available: ', len(physical_devices))
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  0


Load in all the DICOM files and preprocess/label images

In [4]:
# Using the patient_data data structure, load in all the patient data and save it in a dictionary with the folder name as the key
def load_all_patients(path, add_label = False):
    patients = np.array([])
    folder = os.listdir(path)
    # make shuffling and train/test split here
    for name in folder:
        patients= np.append(patients, patient_data.Patient(path, name, add_label))
        if patients[-1].ct_paths == []:
            print(name, "was not processed correctly")
            patients = patients[:-1]
    return patients

patients = load_all_patients(cancerous_path, True)

In [5]:
num_nc = 0
num_c = 0
for i in patients:
    num_nc += sum(1 for j in i.labels if j ==0)   
    num_c += sum(1 for j in i.labels if j ==1)  

print("number of patients loaded:", len(patients))
print("number of non-cancerous images in this dataset:", num_nc)
print("number of cancerous images in this dataset:", num_c)

number of patients loaded: 3
number of non-cancerous images in this dataset: 293
number of cancerous images in this dataset: 53


Setting up train/test data

In [15]:
# mix up the data
patients = shuffle(patients)

# Train-test split should be 80-20. 
# Since the data has been shuffled, we can just grab the 1st 80% of the list and make it the train set and the remainder is the test set
train_patients = patients[:math.floor(len(patients) * 0.8)]
test_patients = patients[math.floor(len(patients) * 0.8):]

def save_imgs(dest, list_paths, list_labels):
    cancerous_img_folder = dest + "/cancerous"
    non_cancerous_img_folder = dest + "/non_cancerous"
    if not os.path.exists(cancerous_img_folder):
        os.makedirs(cancerous_img_folder)
    if not os.path.exists(non_cancerous_img_folder):
        os.makedirs(non_cancerous_img_folder)
    # for our purposes, we know we dont want these folders to have any previous content, so remove anything in here if there is something
    for file in glob.glob(cancerous_img_folder + "/*"):
        os.remove(file)
    for file in glob.glob(non_cancerous_img_folder + "/*"):
        os.remove(file)
    # save images
    for i, dicom_path in enumerate(list_paths):
        if list_labels[i] == 0:
            img_folder = non_cancerous_img_folder
        else:
            img_folder = cancerous_img_folder
        dicom = pydicom.dcmread(dicom_path)
        pix_array = dicom.pixel_array
        img = cv2.normalize(pix_array, None, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite(f"{img_folder}/image_{i}.jpg", img)

train_dest = personal_path + "/train/train"
val_dest = personal_path + "/train/validation"
test_dest = personal_path + "/test"


# save_imgs(train_dest, train_patients)
x_test, y_test = [], []
for patient in test_patients:
    x_test.extend(patient.ct_paths)
    y_test.extend(patient.labels)
save_imgs(test_dest, x_test, y_test)
test_ds = ImageDataGenerator().flow_from_directory(
        directory=test_dest, # the path to the test directory.
        target_size=(512, 512),
        batch_size=32,
        class_mode="binary",
        shuffle = False
    )

# split the train patients up into images and labels
x_train, y_train = [], []
for patient in train_patients:
    x_train.extend(patient.ct_paths)
    y_train.extend(patient.labels)
x_train, y_train = np.array(x_train), np.array(y_train)
x_train, y_train = shuffle(x_train, y_train)

del train_patients
del test_patients
del patients
# del x_test
# del y_test
# del x_train
# del y_train

Create a custom callback to clear any memory that is no longer being used

In [7]:
class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect
        tf.keras.backend.clear_session()

In [ ]:
# Define K-Fold Cross-Validation
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

fold_results = []
AUTOTUNE = tf.data.AUTOTUNE

for fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)):
    print(f"\nTraining fold {fold + 1}/{n_splits}")
    
    x_train_list, y_train_list = x_train[train_idx], y_train[train_idx]
    x_val_list, y_val_list = x_train[val_idx], y_train[val_idx]

    save_imgs(train_dest, x_train_list, y_train_list)
    save_imgs(val_dest, x_val_list, y_val_list)   
        
    # Load the ResNet50 model pre-trained on ImageNet
    model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(512, 512, 3)))
    
    # Custom layers
    flattened = tf.keras.layers.Flatten()(model.output)
    l2 = tf.keras.layers.Dense(128, activation='relu')(flattened)
    l3 = tf.keras.layers.Dense(1, activation='sigmoid')(l2)
    
    # Define the full model
    model = tf.keras.models.Model(inputs=model.input, outputs=l3)
    
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.00001),  # Use a smaller learning rate for end-to-end training
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    train_ds = ImageDataGenerator().flow_from_directory(
            directory=train_dest, # the path to the training directory.
            target_size=(512, 512),
            batch_size=32,
            class_mode="binary",
        )
    val_ds = ImageDataGenerator().flow_from_directory(
            directory=val_dest, # the path to the validation directory.
            target_size=(512, 512),
            batch_size=32,
            class_mode="binary",
        )
    

    # Early stopping and learning rate scheduler
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3)
    
    # Train the model directly using the training and validation data
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=20,
        callbacks=[early_stopping, lr_scheduler],
        verbose=1
    )
    
    # Evaluate the model
    predictions = (model.predict(test_ds) > 0.5).astype("int32")
    report = classification_report(test_ds.classes, predictions, output_dict=True)
    print(classification_report(test_ds.classes, predictions))
    
    # Save fold results
    fold_results.append(report)
    K.clear_session()

# Aggregate results
avg_accuracy = np.mean([fold['accuracy'] for fold in fold_results])
print(f"\nAverage Accuracy Across {n_splits} Folds: {avg_accuracy:.4f}")



Training fold 1/5
Found 188 images belonging to 2 classes.
Found 47 images belonging to 2 classes.


c:\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


c:\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_712']. Received: the structure of inputs=*
  warnings.warn(


In [ ]:
# Define K-Fold Cross-Validation
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
# x_data = np.array(x_c)/255  # Normalize the images
# y_data = np.array(y_c)
# Model training and evaluation loop
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_patients, [0]*len(train_patients))):
    print(f"\nTraining fold {fold + 1}/{n_splits}")
    
    # Split data
    p_train, p_val = train_patients[train_idx], train_patients[val_idx]
    print('point a passed')

#     # possible model to test
#     model = Sequential([
#         Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(512, 512, 1)),
#         MaxPool2D(pool_size=(2, 2), strides=2),
#         Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
#         MaxPool2D(pool_size=(2, 2), strides=2),
#         Conv2D(filters=64, kernel_size=(1, 1), activation='relu', padding='same'),
#         MaxPool2D(pool_size=(2, 2), strides=1),
#         Flatten(),
#         Dense(units=1, activation='sigmoid')
#     ])
    # Build the model
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(512, 512, 1)),
        MaxPool2D(pool_size=(2, 2), strides=2),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(units=1, activation='sigmoid')
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    print('point b passed')

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    x_train = []
    y_train = np.array([])
    for p in p_train:
        x_train.extend(y for y in p.ct.data.values())
        y_train = np.append(y_train, p.labels)
    x_train, y_train = shuffle(x_train, y_train)
    x_train = np.asarray(x_train)

    x_val = []
    y_val = np.array([])
    for p in p_val:
        x_val.extend(y for y in p.ct.data.values())
        y_val = np.append(y_val, p.labels)
    x_val, y_val = shuffle(x_val, y_val)
    x_val = np.asarray(x_val)
    
    datagen = ImageDataGenerator()

    train_generator = datagen.flow(x_train, y_train, batch_size=16)
    val_generator = datagen.flow(x_val, y_val, batch_size=16)

    # Train the model
    model.fit(
        x=x_train,
        y=y_train,
        validation_data=(x_val, y_val),
        batch_size=32,
        epochs=20,
        callbacks=[MyCustomCallback()],
        verbose=1
    )
    print("Model architecture built")
    # Evaluate the model
    predictions = (model.predict(x_test) > 0.5).astype("int32")
    report = classification_report(y_test, predictions, output_dict=True)
    print(classification_report(y_test, predictions))
    
    # Save fold results
    fold_results.append(report)
    K.clear_session()

# Aggregate results
avg_accuracy = np.mean([fold['accuracy'] for fold in fold_results])
print(f"\nAverage Accuracy Across {n_splits} Folds: {avg_accuracy:.4f}")

In [ ]:
print(len(y_test))
for i, j in enumerate(y_test):
    k = predictions[i][0]
    if j != k:
        print(j, k)

Train and test CNN model

In [ ]:

# num_tests = 1
# cnns = []
# for i in range(num_tests):
# cnns.append(cnn.CNN(x_train, x_test, y_train, y_test))

Cross validation and bootstrapping

In [ ]:
# print(cnns[0].test_acc)